In [ ]:
import os,sys
#set the path to the Sandbox repository
sys.path.append('../../../')
#import the sandbox
import sandbox as sb
# set the path to the calibration file
CALIBRATION_FILE = '../../calibration_files/my_calibration.json'


In [ ]:
calib = sb.CalibrationData(file = CALIBRATION_FILE)
sensor = sb.Sensor(calib,sigma_gauss=5)
projector = sb.Projector(calib)

In [ ]:
aruco = sb.ArucoMarkers(sensor, calib)

In [ ]:
module = sb.PrototypingModule(calib,sensor,projector, aruco)

In [ ]:
module.setup()

In [ ]:
module.run()

In [ ]:
module.plot.cmap.set_over('k',1)
module.plot.cmap.set_under('k',1)

In [ ]:
df_position= module.Aruco.aruco_markers
df_position.loc[df_position.is_inside_box == True,('box_x','box_y')].values[0]

### start prototyping 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy


In [ ]:
def gradient_decs_quian(n, frame, xy):
    alpha = 0.01     # 学习率
    tol = 1e-8
    x , y = int(xy[0]), int(xy[1])   # 初始值
    z1 = frame[y][x]
    x_list = [x]
    y_list = [y]
    for i in range(n):
        deriv1, deriv2 = np.gradient(frame)
        #dimension x            
        dx = deriv1[y][x]
        dy = deriv2[y][x]
        
        x = int( x - alpha * dx)
        y = int(y - alpha * dy)
        z2 = frame[y][x]
        x_list.append(x)
        y_list.append(y)
        #if z1 - z2 < tol :
         #   return x_list, y_list
        if z2 < z1:
            z1 = z2
    return x_list, y_list

In [ ]:
def gradient_decs_daniel(n, frame, xy):
    alpha = 0.1     # 学习率
    tol = 1e-15
    x , y = int(xy[0]), int(xy[1])   # 初始值
    z1 = frame[y][x]
    x_list = [x]
    y_list = [y]
    for i in range(n):
        deriv1, deriv2 = np.gradient(frame)
        #dimension x
        if deriv1[y][x] < 0 :
            y_pos = y + 1
        elif deriv1[y][x] > 0:
            y_pos = y -1
        else:
            y_pos = y
        
        if deriv2[y][x] < 0 :
            x_pos = x + 1
        elif deriv2[y][x] > 0:
            x_pos = x - 1
        else:
            x_pos = x
        
        if deriv1[y][x] < deriv2[y][x]:
            y = y_pos
        elif deriv1[y][x] > deriv2[y][x]:
            x = x_pos
        else:
            x= x_pos
            y = y_pos
            
        #z2 = frame[x][y]

        x_list.append(x)
        y_list.append(y)
        
        #if z1 - z2 < tol :
         #   return x_list, y_list, x, y
        #if z2 < z1:
          #  z1 = z2
    return x_list, y_list

In [ ]:
def gradient_decs_interp(n, frame, xy):
    alpha = 0.005# 学习率
    tol = 1e-10
    x , y = int(xy[0]), int(xy[1])   # 初始值
    z1 = frame[y][x]
    z2 =0
    x_list = [x]
    y_list = [y]
    der_y, der_x = np.gradient(frame)
    count = 0
    for i in range(n):
    #while np.abs(z1 - z2) < tol or count <= n:

        x_up = int(np.ceil(x))
        x_down = int(np.floor(x))
        
        if x_up==x_down:
            x_up +=1
        
        y_up = int(np.ceil(y))
        y_down = int(np.floor(y))
        
        if y_up==y_down:
            y_up +=1
        
        points = np.array([[x_up,y_up],
                           [x_up,y_down],
                           [x_down,y_up],
                           [x_down,y_down]])
        values = np.array([frame[y_up][x_up],
                           frame[y_up][x_down],
                           frame[y_down][x_up],
                           frame[y_down][x_down]])
        inter = scipy.interpolate.LinearNDInterpolator(points,values)

        x_ = inter(x_down, y)-inter(x_up, y)
        y_ = inter(x, y_down)-inter(x, y_up)

        dx = x_/(x_down-x_up)
        dy = y_/(y_down - y_up)
   
  
        z1 = inter(x,y)          
        x = ( x - alpha * dx)
        y = (y - alpha * dy)
        z2 = inter(x,y) 
        
        x_list.append(x)
        y_list.append(y)
        count +=1
    #print (count)

    return x_list, y_list

In [ ]:
def plot_from_notebook(sandbox_ax, sandbox_frame, xy):
    frame = sandbox_frame
    ax = sandbox_ax
    frame = calib.s_max - frame
    dx, dy = np.gradient(frame)
    gradient = np.sqrt(dx*dx + dy*dy)
    #ax.pcolormesh(dy, vmin =-2, vmax =2)#, vmin = calib.s_min, vmax = calib.s_max)
    ax.pcolormesh(frame, cmap ="gist_earth")#, vmin = calib.s_min, vmax = calib.s_max)
    levels=np.arange(0,frame.max(), 50)
    ax.contour(frame, levels=levels, colors = 'black' )
    
    if xy is not None:
        ax.plot(xy[0],xy[1],'r*')
        #x_list, y_list, x, y = gradient_decs(1000, frame, xy)
        #x_list1, y_list1 = gradient_decs_quian(1000, frame, xy)
        #x_list2, y_list2 = gradient_decs_daniel(1000, frame, xy)
        x_list3, y_list3 = gradient_decs_interp(5000, frame, xy)

        #ax.plot(x_list1, y_list1,'r*' )
        #ax.plot(x_list1[-1], y_list1[-1],'r*', markersize = 30)
        
        #ax.plot(x_list2, y_list2,'b*' )
        #ax.plot(x_list2[-1], y_list2[-1],'b*', markersize = 30)
        
        ax.plot(x_list3, y_list3,'k*' )
        ax.plot(x_list3[-1], y_list3[-1],'k*', markersize = 30)
    
    


In [ ]:
module.function_to_run = module.plot_sandbox(plot_from_notebook)

In [ ]:
module.active_connection=True

In [ ]:
(module.Aruco.aruco_markers['box_x'].values[0], module.Aruco.aruco_markers['box_y'].values[0])

In [ ]:
int(np.ceil(1.0))

In [ ]:
frame = module.crop_frame(sensor.depth)
frame = calib.s_max - frame
dx, dy = np.gradient(frame)
gradient = np.sqrt(dx**2 + dy**2)
plt.pcolormesh(frame,vmin = frame.min(), vmax = frame.max(), cmap = "gist_earth")


plt.colorbar()

In [ ]:
np.save("topo2.npy", frame)

In [ ]:
gradient

In [ ]:
gradient_decs(1000, frame, (140,100))

In [ ]:
plt.pcolormesh(frame, vmin=calib.s_min, vmax = calib.s_max)
levels=np.arange(calib.s_min, calib.s_max, 50)
plt.contour(frame, levels=levels, colors = 'black' )
plt.colorbar()

In [ ]:
dx